# Training my Model and saving the trained model in the "trained_model.pth" file

mounting the drive

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


importing necessary packages

In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim


importing dataset from csv files

In [4]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/LLM_Model/data/btc_data.csv')
df = df.dropna()

dataframe

In [5]:
df

,timestamp,open,high,low,close,volume,comment_count,sentiment_score,predict,target
0,2020-01-22 07:00:00,8753.66,8818.00,8706.07,8740.27,2896.788689,1.0,0.0,8678.69,0
1,2020-01-22 08:00:00,8740.49,8746.34,8656.00,8678.69,3374.764471,0.0,0.0,8686.28,1
2,2020-01-22 09:00:00,8678.67,8688.99,8643.31,8686.28,1638.063198,0.0,0.0,8663.36,0
3,2020-01-22 10:00:00,8684.56,8686.68,8651.00,8663.36,1303.479686,0.0,0.0,8659.87,0
4,2020-01-22 11:00:00,8662.90,8670.49,8622.00,8659.87,1708.384236,0.0,0.0,8664.50,1
...,...,...,...,...,...,...,...,...,...,...
34516,2023-12-31 19:00:00,42656.69,42695.44,42586.91,42619.04,703.557910,0.0,0.0,42558.01,0
34517,2023-12-31 20:00:00,42619.04,42680.36,42504.56,42558.01,1015.140350,0.0,0.0,42520.72,0
34518,2023-12-31 21:00:00,42558.02,42670.44,42520.72,42520.72,923.438130,0.0,0.0,42257.88,0
34519,2023-12-31 22:00:00,42520.73,42591.10,42056.00,42257.88,1811.594110,0.0,0.0,42283.58,1


adding moving avg to the dataframe for better performance

In [14]:
sequence_length = 24  # using 24 hours of data to predict the next hour

# Calculating moving averages and add them as features
df['ma_5'] = df['close'].rolling(window=5).mean()
df['ma_10'] = df['close'].rolling(window=10).mean()
df['ma_20'] = df['close'].rolling(window=20).mean()

# Droping rows with NaN values created by moving averages
df = df.dropna()

# Updating features
features = df[['open', 'high', 'low', 'close', 'volume', 'comment_count', 'sentiment_score', 'ma_5', 'ma_10', 'ma_20']]
features_scaled = scaler.fit_transform(features)

# Creating sequences
X, y = [], []
for i in range(len(features_scaled) - sequence_length):
    X.append(features_scaled[i:i+sequence_length])
    y.append(features_scaled[i+sequence_length, 3])  # predicting the 'close' price

X, y = np.array(X), np.array(y)



Normalizing the data

In [16]:
scaler = MinMaxScaler()
features_scaled = scaler.fit_transform(features)

splitting the data into train and test set

In [17]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# Create DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

transformer model and parameters

In [18]:
class TransformerModel(nn.Module):
    def __init__(self, input_dim, num_heads, num_layers, hidden_dim):
        super(TransformerModel, self).__init__()
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=input_dim, nhead=num_heads, dim_feedforward=hidden_dim)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)
        self.fc = nn.Linear(input_dim * sequence_length, 1)

    def forward(self, src):
        src = self.transformer_encoder(src)
        src = src.view(src.size(0), -1) 
        output = self.fc(src)
        return output

# Model parameters
input_dim = 10  # number of features
num_heads = 1  # input_dim must be divisible by num_heads
num_layers = 2
hidden_dim = 128

# Initialize model, loss function, and optimizer
model = TransformerModel(input_dim, num_heads, num_layers, hidden_dim)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)



/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


training loop

In [19]:
num_epochs = 50

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i, (inputs, targets) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")


Epoch 1/50, Loss: 0.005807822722585766
Epoch 2/50, Loss: 0.0010043498271704154
Epoch 3/50, Loss: 0.0005206348091627836
Epoch 4/50, Loss: 0.0003402450166270975
Epoch 5/50, Loss: 0.0002688606863970675
Epoch 6/50, Loss: 0.00022832135134947845
Epoch 7/50, Loss: 0.00020277075979892575
Epoch 8/50, Loss: 0.0001666650832545005
Epoch 9/50, Loss: 0.00019420901284506194
Epoch 10/50, Loss: 0.0001344221407182798
Epoch 11/50, Loss: 0.00014224304807493134
Epoch 12/50, Loss: 0.00011183697225109665
Epoch 13/50, Loss: 0.00011504994184957452
Epoch 14/50, Loss: 0.00010720049526959411
Epoch 15/50, Loss: 0.00010434420097241182
Epoch 16/50, Loss: 9.83565493056636e-05
Epoch 17/50, Loss: 0.00011439637016404204
Epoch 18/50, Loss: 8.525860059738868e-05
Epoch 19/50, Loss: 8.218075890592052e-05
Epoch 20/50, Loss: 8.186100734869549e-05
Epoch 21/50, Loss: 8.707736988209024e-05
Epoch 22/50, Loss: 8.186077010578775e-05
Epoch 23/50, Loss: 8.705853616615667e-05
Epoch 24/50, Loss: 6.994566195377231e-05
Epoch 25/50, Loss:

Evaluation

In [21]:
model.eval()
with torch.no_grad():
    test_outputs = model(X_test_tensor)
    test_loss = criterion(test_outputs.squeeze(), y_test_tensor)
    print(f"Test Loss: {test_loss.item()}")


Test Loss: 0.00019228438031859696


Backtesting

In [24]:
def backtest(prices, predictions):
    initial_balance = 1000  # Starting with $1000
    balance = initial_balance
    position = 0  # 0 means no position, 1 means holding BTC

    for i in range(len(predictions) - 1):
        if predictions[i] < prices[i+1] and position == 0:  # Buy signal
            balance -= prices[i+1]
            position = 1
        elif predictions[i] > prices[i+1] and position == 1:  # Sell signal
            balance += prices[i+1]
            position = 0

    # If holding a position at the end, sell it
    if position == 1:
        balance += prices[-1]

    return balance

# Calculate returns
initial_balance = 1000
final_balance = backtest(y_test_tensor.numpy(), test_outputs.numpy().flatten())
returns = (final_balance - initial_balance) / initial_balance
total_return = (final_balance - 1000) / 1000

# Calculate Sharpe Ratio
risk_free_rate = 0.01  # 1% annual risk-free rate
std_dev = np.std(returns)
excess_returns = np.array(returns) - risk_free_rate
sharpe_ratio = np.mean(excess_returns) / np.std(excess_returns) * np.sqrt(365)  # Annualize the Sharpe Ratio

print(f"Final Balance: ${final_balance:.2f}")
print(f"Total Return: {total_return * 100:.2f}%")
print(f"Sharpe Ratio: {sharpe_ratio:.2f}")



Final Balance: $1000.00
Total Return: 0.00%
Sharpe Ratio: -inf


<ipython-input-24-3e1a477b3982>:31: RuntimeWarning: divide by zero encountered in scalar divide
  sharpe_ratio = np.mean(excess_returns) / np.std(excess_returns) * np.sqrt(365)  # Annualize the Sharpe Ratio


saving the trained model

In [20]:
PATH = '/content/drive/MyDrive/Colab Notebooks/trained_model.pth'
torch.save(model.state_dict(), PATH)